In [26]:
#read excell file
import pandas as pd


In [27]:
#read excell file
df = pd.read_excel('./je-d-01.05.04.01.01.xlsx')

In [28]:
#create a new dataframe with the grouped data
df.head()

,Bemerkungen,Unnamed: 1
0,NaN,NaN
1,Zusammensetzung der EU- und Nicht-EU-Länder ge...,NaN
2,NaN,NaN
3,EU-Staaten,Diese Kategorie enthält 27 Länder gemäss aktue...
4,EFTA-Staaten,Diese Kategorie enthält die folgenden Länder: ...


In [25]:
grouped_df

DATA
Kanton Jahr                                                   
Aargau 2011  6293-82947064654362595364566653413946404360113...
       2012  9669172521401189793846791746962725767666052721...
       2013  9816571661041031081271019310468889080757558526...
       2014  1002849115142110120111101102969283856774726187...
       2015  1051650150119108122110114971037395848771696779...
...                                                        ...
Zürich 2016  3727717852241237237537232432931328027824823222...
       2017  3354315149536132631633429128522523721221815517...
       2018  3493516649138633834830028528028124122919617417...
       2019  3590819549740135233732129325126323022021221919...
       2020  3307215444134231632230830924124321721820517816...

[280 rows x 1 columns]